1.   데이터 로딩: CSV 파일에서 데이터를 읽어옵니다.
2.   기본 정보 확인: 데이터의 기본 정보, 처음 몇 개의 행, 기술 통계량을 확인합니다.
3.   결측치 확인 및 처리: 결측치를 확인하고, 수치형 데이터는 평균으로, 범주형
4.    데이터는 최빈값으로 대체합니다.
5.    레이블 확인: 타겟 변수의 분포를 확인하고 시각화합니다.
6.    데이터 인코딩: 범주형 변수를 숫자로 인코딩합니다.
7.    상관관계 분석: 특성 간의 상관관계를 히트맵으로 시각화합니다.
8.    TensorFlow 데이터셋 생성: 전처리된 데이터를 TensorFlow 데이터셋으로 변환합니다.




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
# 1. 데이터 로딩
def load_data(file_path):
    data = pd.read_csv(file_path)
    print("데이터 로드 완료. 데이터 shape:", data.shape)
    return data

In [ ]:
# 2. 기본 정보 확인
def check_basic_info(data):
    print("\n기본 정보:")
    print(data.info())
    print("\n처음 5개 행:")
    print(data.head())
    print("\n기술 통계량:")
    print(data.describe())

In [ ]:
# 3. 결측치 확인 및 처리
def handle_missing_values(data):
    print("\n결측치 확인:")
    print(data.isnull().sum())

    # 결측치 처리 (예: 수치형 데이터는 평균으로, 범주형 데이터는 최빈값으로 대체)
    numeric_features = data.select_dtypes(include=[np.number]).columns
    categorical_features = data.select_dtypes(exclude=[np.number]).columns

    imputer_numeric = SimpleImputer(strategy='mean')
    imputer_categorical = SimpleImputer(strategy='most_frequent')

    data[numeric_features] = imputer_numeric.fit_transform(data[numeric_features])
    data[categorical_features] = imputer_categorical.fit_transform(data[categorical_features])

    print("\n결측치 처리 후:")
    print(data.isnull().sum())

    return data

In [ ]:
# 4. 레이블 확인
def check_labels(data, target_column):
    print(f"\n레이블({target_column}) 확인:")
    print(data[target_column].value_counts())

    plt.figure(figsize=(10, 5))
    data[target_column].value_counts().plot(kind='bar')
    plt.title(f'{target_column} 분포')
    plt.xlabel(target_column)
    plt.ylabel('개수')
    plt.show()

In [ ]:
# 5. 데이터 인코딩
def encode_data(data, target_column):
    # 범주형 변수 인코딩
    le = LabelEncoder()
    for column in data.select_dtypes(include=['object']):
        if column != target_column:  # 타겟 변수는 제외
            data[column] = le.fit_transform(data[column])

    # 타겟 변수 인코딩 (필요한 경우)
    if data[target_column].dtype == 'object':
        data[target_column] = le.fit_transform(data[target_column])

    print("\n인코딩 후 데이터 샘플:")
    print(data.head())

    return data

In [ ]:
# 6. 상관관계 분석
def correlation_analysis(data):
    plt.figure(figsize=(12, 10))
    sns.heatmap(data.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('특성 간 상관관계')
    plt.show()

In [ ]:
# 7. 특성 스케일링
def scale_features(data, target_column):
    scaler = StandardScaler()
    features = data.drop(columns=[target_column])
    scaled_features = scaler.fit_transform(features)

    scaled_data = pd.DataFrame(scaled_features, columns=features.columns)
    scaled_data[target_column] = data[target_column]

    print("\n스케일링 후 데이터 샘플:")
    print(scaled_data.head())

    return scaled_data


In [ ]:
# 메인 함수
def main():
    # 데이터 파일 경로
    file_path = 'your_dataset.csv'

    # 타겟 변수명
    target_column = 'your_target_column'

    # 1. 데이터 로딩
    data = load_data(file_path)

    # 2. 기본 정보 확인
    check_basic_info(data)

    # 3. 결측치 처리
    data = handle_missing_values(data)

    # 4. 레이블 확인
    check_labels(data, target_column)

    # 5. 데이터 인코딩
    data = encode_data(data, target_column)

    # 6. 상관관계 분석
    correlation_analysis(data)

    # 7. 특성 스케일링
    scaled_data = scale_features(data, target_column)

    # TensorFlow 데이터셋으로 변환
    features = scaled_data.drop(columns=[target_column])
    labels = scaled_data[target_column]

    dataset = tf.data.Dataset.from_tensor_slices((features.values, labels.values))
    print("\nTensorFlow 데이터셋 생성 완료")

    return dataset

if __name__ == "__main__":
    dataset = main()